# Demo ShiftAmountActivity
This activity uses a processor to transfer a specified number of objects from an origin resource, which must have a container, to a destination resource, which also must have a container.

Shift Amount checks the possible amount of objects which can be transferred, based on the number of objects available in the origin, the number of objects which can be stored in the destination and the number of objects requested to be transferred. If the number of actually to be transferred objects is zero than an exception is raised. These cases have to be prevented by using appropriate events.

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

In [2]:
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Sites

In [3]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": 10,
    "level": 2,
}



# The two objects used for the simulation
from_site = Site(**data_from_site)


## Creation of Vessel

In [4]:
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

def compute_v_provider(v_empty, v_full):
    return lambda x: 10

data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "geometry": location_from_site,
    "loading_rate": 1,
    "unloading_rate": 1, 
    "capacity": 5,
    "compute_v": compute_v_provider(5, 4.5),
}

hopper = TransportProcessingResource(**data_hopper)

## Definition of Shift Amount activity

In [5]:
shift_amount_activity_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer MP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 100,
    "duration": 10,
}
activity = model.ShiftAmountActivity(**shift_amount_activity_data)

## Run simulation

In [6]:
my_env.run()

In [7]:
display(plot.get_log_dataframe(activity, [activity]))
display(plot.get_log_dataframe(from_site, [activity]))
display(plot.get_log_dataframe(hopper, [activity]))

,Timestamp,Activity,ActivityState
0,1970-01-01 00:00:00,Transfer MP,START
1,1970-01-01 00:00:10,Transfer MP,STOP


,Timestamp,Activity,ActivityState,container level,geometry
0,1970-01-01 00:00:00,Transfer MP,START,2,POINT (4.18055556 52.18664444)
1,1970-01-01 00:00:10,Transfer MP,STOP,0,POINT (4.18055556 52.18664444)


,Timestamp,Activity,ActivityState,geometry,container level
0,1970-01-01 00:00:00,Transfer MP,START,POINT (4.18055556 52.18664444),0.000000
1,1970-01-01 00:00:00,Transfer MP,START,POINT (4.18055556 52.18664444),0.000000
2,1970-01-01 00:00:10,Transfer MP,STOP,POINT (4.18055556 52.18664444),2.000000
3,1970-01-01 00:00:10,Transfer MP,STOP,POINT (4.18055556 52.18664444),2.000000


The resulting levels ob objects in the hopper and the from_site are requested below. 